# Correlation Power Analysis (CPA) Attack on AES-128 — Simulation
**Authors:** Carter, Estelle  
**Project:** Cyber-Physical Systems VIP — Device Security Team

## Overview
This notebook simulates a CPA attack on AES-128 *without physical hardware*. We model power consumption using the **Hamming weight** of the SubBytes output in Round 1, add Gaussian noise, and then recover the key byte-by-byte using Pearson correlation.

### Attack Model
```
Target intermediate value:  SBox[ plaintext[j] ⊕ key[j] ]
Power model:                HammingWeight( intermediate_value ) + noise
Recovery method:            Pearson correlation across all 256 key guesses
```

---
## 0 — Imports & Constants

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

NUM_TRACES = 500       # Number of simulated encryptions
NUM_BYTES  = 16        # AES-128 block size in bytes
NOISE_STD  = 1.0       # Std deviation of Gaussian noise (try: 0.5, 1.0, 2.0, 4.0)

---
## 1 — AES S-Box Lookup Table
The SubBytes step applies this fixed 256-element nonlinear substitution.  
Source: FIPS 197, Section 5.1.1.  

**Nothing to implement here** — just run the cell.

In [ ]:
SBOX = np.array([
    0x63,0x7c,0x77,0x7b,0xf2,0x6b,0x6f,0xc5,0x30,0x01,0x67,0x2b,0xfe,0xd7,0xab,0x76,
    0xca,0x82,0xc9,0x7d,0xfa,0x59,0x47,0xf0,0xad,0xd4,0xa2,0xaf,0x9c,0xa4,0x72,0xc0,
    0xb7,0xfd,0x93,0x26,0x36,0x3f,0xf7,0xcc,0x34,0xa5,0xe5,0xf1,0x71,0xd8,0x31,0x15,
    0x04,0xc7,0x23,0xc3,0x18,0x96,0x05,0x9a,0x07,0x12,0x80,0xe2,0xeb,0x27,0xb2,0x75,
    0x09,0x83,0x2c,0x1a,0x1b,0x6e,0x5a,0xa0,0x52,0x3b,0xd6,0xb3,0x29,0xe3,0x2f,0x84,
    0x53,0xd1,0x00,0xed,0x20,0xfc,0xb1,0x5b,0x6a,0xcb,0xbe,0x39,0x4a,0x4c,0x58,0xcf,
    0xd0,0xef,0xaa,0xfb,0x43,0x4d,0x33,0x85,0x45,0xf9,0x02,0x7f,0x50,0x3c,0x9f,0xa8,
    0x51,0xa3,0x40,0x8f,0x92,0x9d,0x38,0xf5,0xbc,0xb6,0xda,0x21,0x10,0xff,0xf3,0xd2,
    0xcd,0x0c,0x13,0xec,0x5f,0x97,0x44,0x17,0xc4,0xa7,0x7e,0x3d,0x64,0x5d,0x19,0x73,
    0x60,0x81,0x4f,0xdc,0x22,0x2a,0x90,0x88,0x46,0xee,0xb8,0x14,0xde,0x5e,0x0b,0xdb,
    0xe0,0x32,0x3a,0x0a,0x49,0x06,0x24,0x5c,0xc2,0xd3,0xac,0x62,0x91,0x95,0xe4,0x79,
    0xe7,0xc8,0x37,0x6d,0x8d,0xd5,0x4e,0xa9,0x6c,0x56,0xf4,0xea,0x65,0x7a,0xae,0x08,
    0xba,0x78,0x25,0x2e,0x1c,0xa6,0xb4,0xc6,0xe8,0xdd,0x74,0x1f,0x4b,0xbd,0x8b,0x8a,
    0x70,0x3e,0xb5,0x66,0x48,0x03,0xf6,0x0e,0x61,0x35,0x57,0xb9,0x86,0xc1,0x1d,0x9e,
    0xe1,0xf8,0x98,0x11,0x69,0xd9,0x8e,0x94,0x9b,0x1e,0x87,0xe9,0xce,0x55,0x28,0xdf,
    0x8c,0xa1,0x89,0x0d,0xbf,0xe6,0x42,0x68,0x41,0x99,0x2d,0x0f,0xb0,0x54,0xbb,0x16
], dtype=np.uint8)

---
## 2 — Helper: Hamming Weight

Implement a function that returns the number of 1-bits in a byte (or a NumPy array of bytes).  

**Hint:** You can use `bin(x).count('1')` for a scalar, or `np.unpackbits` for a vectorized version.  
A fast approach: build a 256-element lookup table once.

In [ ]:
def hamming_weight(byte_array):
    """
    Compute the Hamming weight (number of 1-bits) for each element.
    
    Args:
        byte_array: np.ndarray of dtype uint8
    Returns:
        np.ndarray of ints, same shape as input
    """
    # TODO: Implement this.
    # Suggestion: precompute HW_LUT = np.array([bin(i).count('1') for i in range(256)])
    #             then return HW_LUT[byte_array]
    pass

---
## 3 — Simulate Trace Collection

This is the core simulation step. You need to:

1. **Generate a random 16-byte AES key** (`true_key`).
2. **Generate `NUM_TRACES` random 16-byte plaintexts.**
3. **For each plaintext, compute the "leaked" power value for each byte position:**
   ```
   intermediate = SBOX[plaintext[j] ^ key[j]]     # AddRoundKey + SubBytes
   power_value  = HammingWeight(intermediate)      # Hamming weight model
   ```
4. **Add Gaussian noise** to each power value.

### Expected outputs:
- `true_key`:   shape `(16,)`, dtype `uint8`  
- `plaintexts`: shape `(NUM_TRACES, 16)`, dtype `uint8`  
- `traces`:     shape `(NUM_TRACES, 16)`, dtype `float64` (HW + noise)

In [ ]:
def generate_traces(num_traces, noise_std):
    """
    Simulate power traces for AES-128 Round 1.
    
    Args:
        num_traces: int, number of encryptions to simulate
        noise_std:  float, standard deviation of Gaussian noise
    Returns:
        true_key:   np.ndarray (16,) uint8
        plaintexts: np.ndarray (num_traces, 16) uint8
        traces:     np.ndarray (num_traces, 16) float64
    """
    # Step 1: Generate random key
    # TODO
    
    # Step 2: Generate random plaintexts
    # TODO
    
    # Step 3: Compute intermediate values  ->  SBOX[plaintext ^ key]
    # TODO  (hint: NumPy XOR and SBOX indexing work element-wise on arrays)
    
    # Step 4: Hamming weight + noise
    # TODO
    
    pass  # return true_key, plaintexts, traces


# Generate and inspect
# true_key, plaintexts, traces = generate_traces(NUM_TRACES, NOISE_STD)
# print(f"True key (hex): {true_key.tobytes().hex()}")
# print(f"Plaintexts shape: {plaintexts.shape}")
# print(f"Traces shape:     {traces.shape}")
# print(f"Trace sample (first 5 values): {traces[0, :5]}")

---
## 4 — CPA Attack: Single Byte

Start with **byte position 0** before generalizing.

For each of the 256 possible key guesses `k`:
1. Compute the hypothetical intermediate: `SBOX[plaintexts[:, 0] ^ k]`
2. Compute the hypothetical Hamming weight for all traces.
3. Compute the **Pearson correlation** between the hypothesis vector and the actual trace values at byte 0.
4. Store the correlation.

The guess with the **highest absolute correlation** is your recovered key byte.

**Hint:** `np.corrcoef(x, y)[0, 1]` gives the Pearson r between two 1-D arrays.

In [ ]:
def attack_single_byte(plaintexts, traces, byte_index):
    """
    Recover a single key byte using CPA.
    
    Args:
        plaintexts: np.ndarray (num_traces, 16) uint8
        traces:     np.ndarray (num_traces, 16) float64
        byte_index: int, which byte position to attack (0-15)
    Returns:
        best_guess:    int (0-255), the recovered key byte
        correlations:  np.ndarray (256,), correlation for each guess
    """
    correlations = np.zeros(256)
    
    for guess in range(256):
        # TODO: compute hypothesis and correlate
        pass
    
    best_guess = np.argmax(np.abs(correlations))
    return best_guess, correlations


# Test on byte 0
# guess, corrs = attack_single_byte(plaintexts, traces, 0)
# print(f"Byte 0 — Recovered: 0x{guess:02x}, Actual: 0x{true_key[0]:02x}, Match: {guess == true_key[0]}")

---
## 5 — Full Key Recovery

Loop `attack_single_byte` over all 16 byte positions. Print each result and compare to the true key.

In [ ]:
def attack_full_key(plaintexts, traces, true_key):
    """
    Recover all 16 bytes of the AES-128 key.
    
    Returns:
        recovered_key: np.ndarray (16,) uint8
    """
    recovered_key = np.zeros(16, dtype=np.uint8)

    for j in range(NUM_BYTES):
        guess, corrs = attack_single_byte(plaintexts, traces, j)
        recovered_key[j] = guess
        match = '✓' if guess == true_key[j] else '✗'
        print(f'  Byte {j:2d} — Recovered: 0x{guess:02x}  Actual: 0x{true_key[j]:02x}  {match}  (corr: {corrs[guess]:.4f})')

    return recovered_key


true_key, plaintexts, traces = generate_traces(NUM_TRACES, NOISE_STD)
print(f'True key (hex): {true_key.tobytes().hex()}\n')
recovered = attack_full_key(plaintexts, traces, true_key)
print(f'\nTrue key:      {true_key.tobytes().hex()}')
print(f'Recovered key: {recovered.tobytes().hex()}')
correct = np.sum(recovered == true_key)
print(f'Correct bytes: {correct}/16  |  Full match: {np.array_equal(true_key, recovered)}')

---
## 6 — Visualization: Correlation by Key Guess

Plot the correlation values for all 256 guesses for a single byte. The correct key byte should show a clear spike.

This is a useful diagnostic — if the spike isn't clear, you may need more traces or less noise.

In [ ]:
# --- Correlation vs Key Guess for Byte 0 ---
guess_0, corrs_0 = attack_single_byte(plaintexts, traces, 0)

fig, ax = plt.subplots(figsize=(10, 4))
ax.bar(range(256), corrs_0, width=1.0, color='steelblue', alpha=0.7)
ax.axvline(x=true_key[0], color='red', linestyle='--', linewidth=2,
           label=f'True key byte: 0x{true_key[0]:02x}')
ax.set_xlabel('Key Guess (0–255)')
ax.set_ylabel('Pearson Correlation')
ax.set_title('CPA — Correlation vs Key Guess (Byte 0)')
ax.legend()
plt.tight_layout()
plt.show()

# --- Heatmap: Correlation across all 16 bytes ---
all_corrs = np.zeros((16, 256))
for j in range(16):
    _, all_corrs[j] = attack_single_byte(plaintexts, traces, j)

fig, ax = plt.subplots(figsize=(12, 5))
im = ax.imshow(np.abs(all_corrs), aspect='auto', cmap='hot', interpolation='nearest')
for j in range(16):
    ax.plot(true_key[j], j, 'c+', markersize=10, markeredgewidth=2)
ax.set_xlabel('Key Guess (0–255)')
ax.set_ylabel('Byte Position')
ax.set_title('CPA — |Correlation| Heatmap (cyan + = true key byte)')
fig.colorbar(im, ax=ax, label='|Pearson r|')
plt.tight_layout()
plt.show()

---
## 7 — Experiment: Traces vs. Recovery Success

This is where you **characterize the information leakage** — directly relevant to the VIP project goals.

Vary `NUM_TRACES` (e.g., 10, 25, 50, 100, 250, 500, 1000) at a fixed noise level and measure how many of the 16 key bytes are correctly recovered. Plot the result.

**Bonus:** Repeat for different `NOISE_STD` values and overlay the curves.

In [ ]:
# --- Experiment: How many traces are needed? ---
trace_counts = [10, 25, 50, 100, 250, 500, 1000]
noise_levels = [0.5, 1.0, 2.0, 4.0]
num_trials = 5  # average over multiple trials for stability

results = {}

for sigma in noise_levels:
    results[sigma] = []
    for n in trace_counts:
        trial_correct = []
        for _ in range(num_trials):
            key, pts, trs = generate_traces(n, sigma)
            rec = attack_full_key(pts, trs, key)
            trial_correct.append(np.sum(rec == key))
        avg_correct = np.mean(trial_correct)
        results[sigma].append(avg_correct)
        print(f'  σ={sigma:.1f}, n={n:5d} -> {avg_correct:.1f}/16 bytes correct (avg of {num_trials} trials)')

# --- Plot ---
fig, ax = plt.subplots(figsize=(10, 5))
colors = ['#1f77b4', '#ff7f0e', '#2ca02c', '#d62728']

for i, sigma in enumerate(noise_levels):
    ax.plot(trace_counts, results[sigma], 'o-', color=colors[i],
            label=f'σ = {sigma}', linewidth=2, markersize=6)

ax.axhline(y=16, color='gray', linestyle=':', alpha=0.5, label='Full recovery (16/16)')
ax.set_xlabel('Number of Traces')
ax.set_ylabel('Correct Key Bytes (out of 16)')
ax.set_title('CPA Success vs. Trace Count at Different Noise Levels')
ax.set_xscale('log')
ax.set_ylim(-0.5, 17)
ax.legend()
ax.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

---
## 8 — Discussion & Next Steps

Once you've completed the above, document your observations here:

- How many traces were needed for full recovery at each noise level?
- How does this compare to theoretical expectations?
- What would change with real hardware traces? (alignment, non-Gaussian noise, algorithmic noise)
- How could countermeasures (masking, shuffling) break this attack?
- Connection to timing attack: what's the same principle, and what's different?

In [ ]:
# --- Record your observations below ---

# 1. TRACE THRESHOLD
#    At σ=0.5, full recovery needed ~___ traces.
#    At σ=1.0, it needed ~___.
#    At σ=4.0, it needed ~___.
#    Rough scaling: traces needed ∝ σ² (why? think about SNR).

# 2. THEORETICAL COMPARISON
#    For CPA, the correlation ρ scales as 1/√(n) where n = num traces.
#    The signal is fixed (HW range 0-8), noise is σ.
#    So we need n ∝ σ² to maintain the same SNR — does your data match?

# 3. REAL HARDWARE vs. SIMULATION
#    - Trace alignment: real oscilloscope traces need clock-cycle alignment (jitter)
#    - Noise model: real noise isn't purely Gaussian — includes quantization,
#      power supply ripple, and algorithmic noise from other AES operations
#    - Leakage model: Hamming weight is an approximation; real chips may leak
#      Hamming distance (bits flipped from previous state) instead
#    - Multiple time samples: real traces have thousands of samples per encryption,
#      need to identify which sample(s) correspond to the target operation

# 4. COUNTERMEASURES
#    - Boolean masking: compute SBox(p ⊕ k ⊕ m) with random mask m,
#      breaks the correlation between key guess and power consumption
#    - Shuffling: randomize the order of byte processing across rounds,
#      dilutes correlation across time samples
#    - Hiding: add random delays or dummy operations to decouple
#      power from data
#    → Higher-order attacks (2nd order DPA) can still defeat 1st-order masking

# 5. CONNECTION TO TIMING ATTACK (main.rs)
#    Same:  both exploit data-dependent physical observables to recover secrets
#           byte-by-byte (divide and conquer).
#    Different: timing attack exploits control flow (early exit),
#              power attack exploits data processing (bit transitions).
#    Timing fix (constant-time compare) does NOT prevent power analysis.